<a href="https://colab.research.google.com/github/StuartLiv/CPSC-440-Project/blob/main/models/hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.transforms.functional as F

In [2]:
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array # TODO don't use keras
from keras.utils import load_img
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import random

from google.colab import userdata

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


# 1. Set up project in runtime
Copy the following cell and run it with your credentials to clone the repo, getting datasets as runtime files, and then delete your credentials.

In [3]:
!git config --global user.email {userdata.get('email')}
!git config --global user.name {userdata.get('name')}
!git clone https://{userdata.get('token')}@github.com/StuartLiv/CPSC-440-Project
%pwd

fatal: destination path 'CPSC-440-Project' already exists and is not an empty directory.


'/content'

# 2. Preprocess data

Datasets:
- `medset_multisize`
  - Single datasets. Dataset construction code in repo too
  - 12000 Images, in randomized order
  - Res 256x256 (grayscale images also available in 128x128, 64x64, 32x32)


In [4]:
def make_image_arr(path,cut=None,end=False):

  def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

  files = os.listdir(path)
  files = sorted_alphanumeric(files)

  if end:
    files.reverse()
  if cut is not None:
    files = files[:cut]

  arr = []
  for i in tqdm(files):
    img = load_img(path + '/'+i)
    arr.append(img_to_array(img) / 255)

  return np.array(arr)

In [5]:
graynp = make_image_arr('/content/CPSC-440-Project/datasets/medset_multisize/gray64',cut=100,end=True)

# take one axis
graynp = np.moveaxis(graynp, [3,1], [1,2])

# convert to LAB scale
graynp = graynp[:, 0:1, :, :] * 100

# tensor to GPU
gray = torch.from_numpy(graynp).to(device)

100%|██████████| 100/100 [00:00<00:00, 3533.53it/s]


# 3. Model
Model generated below:

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential()
        self.decoder = nn.Sequential()
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Create the autoencoder
autoencoder = Autoencoder()

Train Model (SGD)

In [7]:
autoencoder.encoder = torch.load('/content/CPSC-440-Project/models/old models/lab_mle_encoder_64')
autoencoder.decoder = torch.load('/content/CPSC-440-Project/models/old models/lab_mle_decoder')
autoencoder.eval()


Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_p

## Hook
Add hooks to self.encoder or self.decoder, with the indices listed by `.eval` above!

In [8]:

# a dict to store the activations
activation = {}
def getActivation(name):
  # the hook signature
  def hook(model, input, output):
    activation[name] = output.detach().cpu().numpy()
  return hook

# register forward hooks on the layers with the following dims:
dims = [32,64,128,256]
autoencoder.encoder[-1].register_forward_hook(getActivation('en'))
autoencoder.decoder[0].register_forward_hook(getActivation('de64'))
autoencoder.decoder[6].register_forward_hook(getActivation('de128'))
autoencoder.decoder[12].register_forward_hook(getActivation('de256'))

# h3 = autoencoder.decoder.register_forward_hook(getActivation('sigmoid')) # this is excatly the output!


In [9]:
def get_rank(S,cut):
  sum = np.sum(S)
  i = 0
  while np.sum(S[:i]) < cut * np.sum(S[i:]):
    i += 1
  return i

In [10]:
n = 100
ranks = np.zeros((3,n))
count = 0
for i in tqdm(np.random.randint(0, len(gray), n)):
  # Colorize the image
  t = gray[i].unsqueeze(0).to(device)
  autoencoder(t)

  for j in range(3):
    key = list(activation.keys())[j]
    dim = dims[j]
    # print(activation[key][0].shape)
    U,S,V = np.linalg.svd(activation[key][0].reshape(-1,dim**2))
    # print(S.shape)
    ranks[j,count] = get_rank(S,3)

  count += 1

100%|██████████| 100/100 [15:53<00:00,  9.53s/it]


In [16]:
ranks

array([[87., 83., 83., 92., 76., 11., 71., 83., 83., 85., 91., 76., 94.,
        59., 63., 88., 88., 63., 80., 90., 79., 57., 37., 75., 12., 75.,
        87., 76., 68., 76., 78., 89., 63., 73., 83., 67., 68., 13., 68.,
        73., 77., 95., 93., 89., 57., 92., 73.,  8., 92., 91., 50., 73.,
        77., 87., 77., 77., 77., 58., 51., 69., 83., 76., 77., 92., 87.,
        37., 91., 37., 87., 85., 89., 82., 68., 92., 86., 53., 78., 24.,
        57., 82., 13., 88., 82., 82., 89., 86., 34., 76., 67., 87., 89.,
        67., 80., 12., 77., 77., 34., 58., 86., 82.],
       [20., 19., 16., 17., 12.,  4., 13., 17., 19., 17., 18., 12., 17.,
         7., 14., 18., 18., 14., 15., 17., 16., 14.,  7., 14.,  4., 16.,
        17., 17., 14., 13., 16., 18., 14., 11., 17., 12.,  7.,  5.,  7.,
        11., 12., 19., 15., 18., 13., 17., 12.,  4., 15., 18.,  6., 11.,
        13., 18., 12., 12., 16., 13.,  8.,  9., 15., 12., 16., 17., 17.,
         7., 18.,  7., 14., 18., 18., 15.,  7., 15., 18.,  8., 15., 10

In [17]:
np.save('lab64_ranks',ranks)

In [ ]:
np.load('')

In [18]:
for i in range(3):
  print(f'{np.mean(ranks[i])} ± {np.sqrt(np.var(ranks,axis=1))[i]:.2f}')

71.55 ± 21.19
13.62 ± 4.31
11.64 ± 0.93


# 4. Push changes

Look for any changes in the project directory, excluding this notebook, and push them.


# 4. Save notebook

Commiting this notebook requires a special maneuver:

> File > Save a copy in github > enter original `path` + new commit msg

That's it!